In [1]:
import pandas as pd
import numpy as np
from random import randint

# path to files
dataPath = 'C:/projects/imove/silomatsim/muc/pythonProcessing/'

# helper methods
def RandomHead(df, size):
    randomHead = randint(size, len(df))
    return df.head(randomHead)[randomHead-size:randomHead]
def XLS2CSV(filepath, filename, sheet=0):
    df = pd.read_excel(filepath+filename+'.xlsx', sheet_name=sheet)
    df.to_csv(filepath+filename+'.csv', encoding='utf-8', index=False)
def moveCol(df, colName, toPos):
    col = df[colName]
    df = df.drop(columns={colName})
    df.insert(toPos, colName, col)
    return df
def mergeColumns(df, newColumnName, columnNames):
    # assign value from first column
    df[newColumnName] = df[columnNames[0]]
    # add values from the other columns
    for i in range(1, len(columnNames)):
        df[newColumnName] = df[newColumnName] + df[columnNames[i]]
    # drop merged columns
    df = df.drop(columns=columnNames)
    return df

# SA4 to SA1

In [2]:
# ------------------------------------------------------  Read MB Data
# This will be used to match the SA1s with SA4s
def readMB2011():
    # columns to use
    columns = {
        'SA1_7DIGITCODE_2011': int,
        'SA4_NAME_2011': str,
    }
    # read the data
    df = pd.read_csv(dataPath + 'in_MB_2011_WA.csv', usecols = columns.keys(), dtype = columns)
    # rename the columns
    df = df.rename(columns={'SA1_7DIGITCODE_2011': 'SA1'})
    # drop duplicates
    df = df.drop_duplicates(subset=['SA1'])
    return df

# reads SA1s for all WA
mbwaDF = readMB2011()
RandomHead(mbwaDF, 5)

,SA1,SA4_NAME_2011
18713,5117423,Perth - South West
18726,5116803,Perth - South West
18729,5117103,Perth - South West
18731,5117102,Perth - South West
18732,5117104,Perth - South West


In [3]:
# ------------------------------------------------------  Assign area codes & Filter out areas to just Perth
def AssignAreaCodes2011(df):
    # convert the specifed column into array
    columnArray = df['SA4_NAME_2011'].values
    columnLength = len(columnArray)
    areaCodeCol = [-1] * columnLength

    # corresponding regions to area codes
    geoAreas = {
        'Perth - Inner': 49,
        'Perth - North East': 49,
        'Perth - North West': 50,
        'Perth - South East': 51,
        'Perth - South West': 52,
    }

    # assign area code to the corresponding region
    for index in range(columnLength):
        value = columnArray[index]
        if value in geoAreas:
            areaCodeCol[index] = geoAreas[value]

    # append new area code column
    df['SA4'] = areaCodeCol

    # drop all rows that are not equal to -1
    df = df[df.SA4 != -1]
    
    return df

mbperthDF = AssignAreaCodes2011(mbwaDF)
mbperthDF = mbperthDF.drop(columns='SA4_NAME_2011')
RandomHead(mbperthDF, 5)

,SA1,SA4
16871,5114109,51
16872,5114128,51
16873,5114031,51
16874,5113832,51
16876,5113831,51


# People

## Count

In [4]:
# ------------------------------------------------------  Read Gender Data
def readPeopleCount2011():
    # columns to use
    columns = {
        'SA1': int,
        'Count': int,
    }
    # read the data
    df = pd.read_csv(dataPath + 'in_TB_P_SA1_2011.csv', usecols = columns.keys(), dtype = columns)
    return df
    
# assign values from MB_WA to people = correlate regions to people
peopleCountDF = readPeopleCount2011()
RandomHead(peopleCountDF, 5)

,SA1,Count
651,5105409,425
652,5105410,216
653,5105411,404
654,5105412,251
655,5105413,394


## Gender

In [5]:
# ------------------------------------------------------  Read Gender Data
def readGender2011():
    # columns to use
    columns = {
        'SA1': int,
        'Male': int,
        'Female': int
    }
    # read the data
    df = pd.read_csv(dataPath + 'in_GenderBySA1_2011.csv', usecols = columns.keys(), dtype = columns)
    return df
    
# assign values from MB_WA to people = correlate regions to people
genderDF_1 = pd.merge(readGender2011(), mbperthDF, on=['SA1'], how='left')
# re-arrange columns
genderDF_1 = moveCol(genderDF_1, 'SA4', 1)

RandomHead(genderDF_1, 5)

,SA1,SA4,Male,Female
3788,5119010,52,200,199
3789,5119011,52,249,254
3790,5119012,52,200,215
3791,5119013,52,223,201
3792,5119014,52,167,170


## Age Groups

In [6]:
# ------------------------------------------------------  Read Age Group Data
def readAgeGroups2011():
    # columns to use
    columns = {
        'SA1': int,
        '0-4 years': int,
        '5-9 years': int,
        '10-14 years': int,
        '15-19 years': int,
        '20-24 years': int,
        '25-29 years': int,
        '30-34 years': int,
        '35-39 years': int,
        '40-44 years': int,
        '45-49 years': int,
        '50-54 years': int,
        '55-59 years': int,
        '60-64 years': int,
        '65-69 years': int,
        '70-74 years': int,
        '75-79 years': int,
        '80-84 years': int,
        '85-89 years': int,
        '90-94 years': int,
        '95-99 years': int,
        '100 years and over': int
    }
    # read the data
    df = pd.read_csv(dataPath + 'in_AgeGroupsBySA1_2011.csv', usecols = columns.keys(), dtype = columns)
    return df
    
ageGroupsDF = readAgeGroups2011()
ageGroupsDF.head(2)

,SA1,0-4 years,5-9 years,10-14 years,15-19 years,20-24 years,25-29 years,30-34 years,35-39 years,40-44 years,...,55-59 years,60-64 years,65-69 years,70-74 years,75-79 years,80-84 years,85-89 years,90-94 years,95-99 years,100 years and over
0,5103001,0,0,0,0,0,3,0,3,3,...,6,4,7,11,30,62,54,30,5,3
1,5103002,11,21,38,44,24,10,9,9,21,...,16,20,15,21,14,11,0,0,0,0


In [7]:
ageGroupsDF_1 = ageGroupsDF

# merge columns
ageGroupsDF_1 = mergeColumns(ageGroupsDF_1, '85 years and over', [
    '85-89 years',
    '90-94 years',
    '95-99 years',
    '100 years and over'
])

# rename
ageGroupsDF_1 = ageGroupsDF_1.rename(columns={
    '0-4 years': 'age_1',
    '5-9 years': 'age_2',
    '10-14 years': 'age_3',
    '15-19 years': 'age_4',
    '20-24 years': 'age_5',
    '25-29 years': 'age_6',
    '30-34 years': 'age_7',
    '35-39 years': 'age_8',
    '40-44 years': 'age_9',
    '45-49 years': 'age_10',
    '50-54 years': 'age_11',
    '55-59 years': 'age_12',
    '60-64 years': 'age_13',
    '65-69 years': 'age_14',
    '70-74 years': 'age_15',
    '75-79 years': 'age_16',
    '80-84 years': 'age_17',
    '85 years and over': 'age_18'
})

RandomHead(ageGroupsDF_1, 5)

,SA1,age_1,age_2,age_3,age_4,age_5,age_6,age_7,age_8,age_9,age_10,age_11,age_12,age_13,age_14,age_15,age_16,age_17,age_18
264,5104107,13,8,5,21,100,149,63,28,21,14,14,16,10,11,0,0,3,0
265,5104108,52,11,19,52,159,274,222,184,152,177,152,125,112,62,29,20,7,3
266,5104109,32,3,6,23,115,284,291,219,212,186,158,160,111,46,15,6,3,3
267,5104110,28,11,16,17,112,236,172,129,132,126,120,88,70,47,24,13,0,6
268,5104111,29,5,24,86,271,425,342,231,224,226,233,197,151,140,97,54,45,78


## Income

In [8]:
def readIncomeGroups2011():
    # columns to use
    columns = {
        'SA1': int,
        'Negative income':int,
        'Nil income': int,
        '$1-$199 ($1-$10,399)': int,
        '$200-$299 ($10,400-$15,599)': int,
        '$300-$399 ($15,600-$20,799)': int,
        '$400-$599 ($20,800-$31,199)': int,
        '$600-$799 ($31,200-$41,599)': int,
        '$800-$999 ($41,600-$51,999)': int,
        '$1,000-$1,249 ($52,000-$64,999)': int,
        '$1,250-$1,499 ($65,000-$77,999)': int,
        '$1,500-$1,999 ($78,000-$103,999)': int,
        '$2,000 or more ($104,000 or more)': int,
        'Not stated': int,
        'Not applicable': int,
        'Overseas visitor': int,
    }
    # read the data
    df = pd.read_csv(dataPath + 'in_TB_WeeklyIncomeBySA1_2011.csv', usecols = columns.keys(), dtype = columns)
    
    # rename columns
    df = df.rename(columns={
        'Negative income': 'Negative',
        'Nil income': 'Nil',
        '$1-$199 ($1-$10,399)': '1-199 (1-10399)',
        '$200-$299 ($10,400-$15,599)': '200-299 (10400-15599)',
        '$300-$399 ($15,600-$20,799)': '300-399 (15600-20799)',
        '$400-$599 ($20,800-$31,199)': '400-599 (20800-31199)',
        '$600-$799 ($31,200-$41,599)': '600-799 (31200-41599)',
        '$800-$999 ($41,600-$51,999)': '800-999 (41600-51999)',
        '$1,000-$1,249 ($52,000-$64,999)': '1000-1249 (52000-64999)',
        '$1,250-$1,499 ($65,000-$77,999)': '1250-1499 (65000-77999)',
        '$1,500-$1,999 ($78,000-$103,999)': '1500-1999 (78000-103999)',
        '$2,000 or more ($104,000 or more)': '2000-more (104000-more)'
    })
    
    # rename
    df = df.rename(columns={
        'Negative': 'income_1',
        'Nil': 'income_2',
        'Not stated': 'income_13',
        'Not applicable': 'income_14',
        'Overseas visitor': 'income_15',
        '1-199 (1-10399)': 'income_3',
        '200-299 (10400-15599)': 'income_4',
        '300-399 (15600-20799)': 'income_5',
        '400-599 (20800-31199)': 'income_6',
        '600-799 (31200-41599)': 'income_7',
        '800-999 (41600-51999)': 'income_8',
        '1000-1249 (52000-64999)': 'income_9',
        '1250-1499 (65000-77999)': 'income_10',
        '1500-1999 (78000-103999)': 'income_11', 
        '2000-more (104000-more)': 'income_12'
    })
    
    return df
    
incomeGrupsDF = readIncomeGroups2011()
incomeGrupsDF.head(5)

,SA1,income_1,income_2,income_3,income_4,income_5,income_6,income_7,income_8,income_9,income_10,income_11,income_12,income_13,income_14,income_15
0,5103001,0,3,6,17,38,42,28,20,14,6,6,14,31,3,5
1,5103002,0,43,18,14,14,33,9,10,25,12,25,63,10,67,8
2,5103003,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,5103004,0,19,15,0,0,3,0,3,3,0,0,0,0,30,0
4,5103005,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Industry of Employment

In [9]:
def readIndustryGroups2011():
    # columns to use
    columns = {
        'SA1': int,
        'Agriculture, Forestry and Fishing': int,
        'Mining': int,
        'Manufacturing': int,
        'Electricity, Gas, Water and Waste Services': int,
        'Construction': int,
        'Wholesale Trade': int,
        'Retail Trade': int,
        'Accommodation and Food Services': int,
        'Transport, Postal and Warehousing': int,
        'Information Media and Telecommunications': int,
        'Financial and Insurance Services': int,
        'Rental, Hiring and Real Estate Services': int,
        'Professional, Scientific and Technical Services': int,
        'Administrative and Support Services': int,
        'Public Administration and Safety': int,
        'Education and Training': int,
        'Health Care and Social Assistance': int,
        'Arts and Recreation Services': int,
        'Other Services': int,
        'Inadequately described': int,
        'Not stated': int,
        'Not applicable': int,
        'Overseas visitor': int
    }
    # read the data
    df = pd.read_csv(dataPath + 'in_TB_P_IND_SA1_2011.csv', usecols = columns.keys(), dtype = columns)    
    
        # rename
    df = df.rename(columns={
        'Agriculture, Forestry and Fishing': 'industry_1',
        'Mining': 'industry_2',
        'Manufacturing': 'industry_3',
        'Electricity, Gas, Water and Waste Services': 'industry_4',
        'Construction': 'industry_5',
        'Wholesale Trade': 'industry_6',
        'Retail Trade': 'industry_7',
        'Accommodation and Food Services': 'industry_8',
        'Transport, Postal and Warehousing': 'industry_9',
        'Information Media and Telecommunications': 'industry_10',
        'Financial and Insurance Services': 'industry_11',
        'Rental, Hiring and Real Estate Services': 'industry_12',
        'Professional, Scientific and Technical Services': 'industry_13',
        'Administrative and Support Services': 'industry_14',
        'Public Administration and Safety': 'industry_15',
        'Education and Training': 'industry_16',
        'Health Care and Social Assistance': 'industry_17',
        'Arts and Recreation Services': 'industry_18',
        'Other Services': 'industry_19',
        'Inadequately described': 'industry_20',
        'Not stated': 'industry_21',
        'Not applicable': 'industry_22',
        'Overseas visitor': 'industry_23'
    })
    
    return df
    
industryGrupsDF = readIndustryGroups2011()
industryGrupsDF.head(5)

,SA1,industry_1,industry_2,industry_3,industry_4,industry_5,industry_6,industry_7,industry_8,industry_9,...,industry_14,industry_15,industry_16,industry_17,industry_18,industry_19,industry_20,industry_21,industry_22,industry_23
0,5103001,0,5,0,0,0,0,0,0,3,...,0,0,0,5,0,0,0,0,212,5
1,5103002,0,11,10,3,10,5,17,0,3,...,4,9,18,18,3,3,3,0,182,8
2,5103003,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,5103004,0,0,0,0,3,0,0,3,0,...,0,0,7,0,0,0,0,0,62,0
4,5103005,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Combine

In [10]:
peopleCombinedDF_1 = pd.merge(genderDF_1, ageGroupsDF_1, on=['SA1'], how='left') # gender & age
peopleCombinedDF_2 = pd.merge(peopleCombinedDF_1, incomeGrupsDF, on=['SA1'], how='left') # & income
peopleCombinedDF_3 = pd.merge(peopleCombinedDF_2, industryGrupsDF, on=['SA1'], how='left') # & indusry of employment
peopleCombinedDF_4 = pd.merge(peopleCombinedDF_3, peopleCountDF, on=['SA1'], how='left') # & count per SA1

peopleMasterDF = peopleCombinedDF_4

# sum seperate properties
peopleMasterDF['sex_total'] = peopleMasterDF.loc[:,'Male':'Female'].sum(axis = 1)
peopleMasterDF['age_total'] = peopleMasterDF.loc[:,'age_1':'age_18'].sum(axis = 1)
peopleMasterDF['income_total'] = peopleMasterDF.loc[:,'income_1':'income_15'].sum(axis = 1)
peopleMasterDF['industry_total'] = peopleMasterDF.loc[:,'industry_1':'industry_23'].sum(axis = 1)

RandomHead(peopleMasterDF, 15)

,SA1,SA4,Male,Female,age_1,age_2,age_3,age_4,age_5,age_6,...,industry_19,industry_20,industry_21,industry_22,industry_23,Count,sex_total,age_total,income_total,industry_total
533,5104737,49,252,244,28,27,37,27,25,37,...,11,8,0,247,5,491,496,503,488,511
534,5104738,49,307,288,51,42,38,32,38,42,...,9,3,0,315,3,597,595,599,604,589
535,5104739,49,256,275,49,29,31,20,37,43,...,8,3,0,274,3,535,531,526,528,526
536,5104740,49,210,204,37,23,23,18,32,25,...,12,0,3,209,9,418,414,425,418,409
537,5104741,49,284,295,40,24,28,37,30,37,...,7,3,0,293,0,574,579,571,570,575
538,5104742,49,221,224,26,15,25,48,37,23,...,9,3,3,203,3,446,445,445,453,436
539,5104743,49,298,294,24,33,47,60,45,34,...,12,4,0,281,3,590,592,593,599,590
540,5104744,49,196,279,16,18,27,20,31,13,...,14,0,3,268,0,475,475,477,465,481
541,5104745,49,214,186,24,24,27,37,24,14,...,9,3,0,197,3,400,400,397,408,387
542,5104801,49,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
peopleMasterDF.to_csv(dataPath+'out_TB_peopleBySA1_2011.csv', index=False)
print("CSV created.")

CSV created.


# Dwellings

## Total

In [12]:
def readDwellingsTotal2011():
    # columns to use
    columns = {
        'SA1': int,
        'Count':int,
    }
    # read the data
    df = pd.read_csv(dataPath + 'in_TB_OPD_SA1.csv', usecols = columns.keys(), dtype = columns)
        
    return df
    
dwellsTotalDF = readDwellingsTotal2011()
dwellsTotalDF.head(5)

,SA1,Count
0,5103001,180
1,5103002,113
2,5103003,0
3,5103004,5
4,5103005,0


In [13]:
# assign values from MB_WA to people = correlate regions to people
dwellsTotalDF_1 = pd.merge(dwellsTotalDF, mbperthDF, on=['SA1'], how='left')
# order the column
dwellsTotalDF_1 = moveCol(dwellsTotalDF_1, 'SA4', 1)
# display
RandomHead(dwellsTotalDF_1, 5)

,SA1,SA4,Count
1093,5107502,50,118
1094,5107503,50,128
1095,5107504,50,192
1096,5107505,50,115
1097,5107506,50,220


## Structure

In [14]:
def readDwellingsStructure2011():
    # columns to use
    columns = {
        'SA1': int,
        'Separate house': int,
        'Semi-detached, row or terrace house, townhouse etc with one storey': int,
        'Semi-detached, row or terrace house, townhouse etc with two or more storeys': int,
        'Flat, unit or apartment in a one or two storey block': int,
        'Flat, unit or apartment in a three storey block': int,
        'Flat, unit or apartment in a four or more storey block': int,
        'Flat, unit or apartment attached to a house': int,
        'Caravan, cabin, houseboat': int,
        'Improvised home, tent, sleepers out': int,
        'House or flat attached to a shop, office, etc.': int,
        'Not stated': int,
        'Not applicable': int
    }
    # read the data
    df = pd.read_csv(dataPath + 'in_TB_OPD_STR_SA1.csv', usecols = columns.keys(), dtype = columns)
    
    return df
    
dwellsStructureDF = readDwellingsStructure2011()
dwellsStructureDF.head(5)

,SA1,Separate house,"Semi-detached, row or terrace house, townhouse etc with one storey","Semi-detached, row or terrace house, townhouse etc with two or more storeys","Flat, unit or apartment in a one or two storey block","Flat, unit or apartment in a three storey block","Flat, unit or apartment in a four or more storey block","Flat, unit or apartment attached to a house","Caravan, cabin, houseboat","Improvised home, tent, sleepers out","House or flat attached to a shop, office, etc.",Not stated,Not applicable
0,5103001,12,46,76,42,0,0,0,0,0,0,0,0
1,5103002,113,0,0,0,0,0,0,0,0,0,0,0
2,5103003,0,0,0,0,0,0,0,0,0,0,0,0
3,5103004,0,0,5,0,0,0,0,0,0,0,0,0
4,5103005,0,0,0,0,0,0,0,0,0,0,0,0


In [15]:
# assign values from MB_WA to people = correlate regions to people
# dwellsStructureDF_1 = pd.merge(dwellsStructureDF, mbperthDF, on=['SA1'], how='left')
dwellsStructureDF_1 = dwellsStructureDF

# merge columns
dwellsStructureDF_1 = mergeColumns(dwellsStructureDF_1, 'Semi-detached', [
    'Semi-detached, row or terrace house, townhouse etc with two or more storeys',
    'Semi-detached, row or terrace house, townhouse etc with one storey'
])
dwellsStructureDF_1 = mergeColumns(dwellsStructureDF_1, 'Flat, unit, apartment', [
    'Flat, unit or apartment attached to a house',
    'Flat, unit or apartment in a four or more storey block',
    'Flat, unit or apartment in a three storey block',
    'Flat, unit or apartment in a one or two storey block'
])
dwellsStructureDF_1 = mergeColumns(dwellsStructureDF_1, 'Improvised', [
    'Caravan, cabin, houseboat',
    'House or flat attached to a shop, office, etc.',
    'Improvised home, tent, sleepers out'
])

# rename columns
dwellsStructureDF_1 = dwellsStructureDF_1.rename(columns={
    'Separate house': 'structure_1',
    'Semi-detached': 'structure_2',
    'Flat, unit, apartment': 'structure_3',
    'Improvised': 'structure_4',
    'Not stated': 'structure_5',
    'Not applicable': 'structure_6',
})

# order the columns
# dwellsStructureDF_1 = moveCol(dwellsStructureDF_1, 'SA4', 1)
dwellsStructureDF_1 = moveCol(dwellsStructureDF_1, 'structure_1', 1)
dwellsStructureDF_1 = moveCol(dwellsStructureDF_1, 'structure_2', 2)
dwellsStructureDF_1 = moveCol(dwellsStructureDF_1, 'structure_3', 3)
dwellsStructureDF_1 = moveCol(dwellsStructureDF_1, 'structure_4', 4)
dwellsStructureDF_1 = moveCol(dwellsStructureDF_1, 'structure_5', 5)
dwellsStructureDF_1 = moveCol(dwellsStructureDF_1, 'structure_6', 6)

RandomHead(dwellsStructureDF_1, 5)

,SA1,structure_1,structure_2,structure_3,structure_4,structure_5,structure_6
480,5104612,128,10,31,0,0,0
481,5104613,88,90,38,0,0,0
482,5104614,33,65,223,0,0,0
483,5104615,62,64,102,0,0,0
484,5104616,74,103,109,0,0,0


## Beds

In [16]:
def readDwellingsBed2011():
    # columns to use
    columns = {
        'SA1': int,
        'None (includes bedsitters)': int,
        '1 bedroom': int,
        '2 bedrooms': int,
        '3 bedrooms': int,
        '4 bedrooms': int,
        '5 bedrooms': int,
        '6 bedrooms': int,
        '7 bedrooms': int,
        '8 bedrooms': int,
        '9 bedrooms': int,
        '10 bedrooms': int,
        '11 bedrooms': int,
        '12 bedrooms': int,
        '13 bedrooms': int,
        '14 bedrooms': int,
        '15 bedrooms': int,
        '16 bedrooms': int,
        '17 bedrooms': int,
        '18 bedrooms': int,
        '19 bedrooms': int,
        '20 bedrooms': int,
        '21 bedrooms': int,
        '22 bedrooms': int,
        '23 bedrooms': int,
        '24 bedrooms': int,
        '25 bedrooms': int,
        '26 bedrooms': int,
        '27 bedrooms': int,
        '28 bedrooms': int,
        '29 bedrooms': int,
        '30 or more bedrooms': int,
        'Not stated': int,
        'Not applicable': int

    }
    # read the data
    df = pd.read_csv(dataPath + 'in_TB_OPD_BED_SA1.csv', usecols = columns.keys(), dtype = columns)
        
    return df
    
dwellsBedDF = readDwellingsBed2011()
dwellsBedDF.head(5)

,SA1,None (includes bedsitters),1 bedroom,2 bedrooms,3 bedrooms,4 bedrooms,5 bedrooms,6 bedrooms,7 bedrooms,8 bedrooms,...,23 bedrooms,24 bedrooms,25 bedrooms,26 bedrooms,27 bedrooms,28 bedrooms,29 bedrooms,30 or more bedrooms,Not stated,Not applicable
0,5103001,0,54,94,14,3,0,0,0,0,...,0,0,0,0,0,0,0,0,15,0
1,5103002,0,0,3,24,58,23,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5103003,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,5103004,0,0,5,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5103005,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
# assign values from MB_WA to people = correlate regions to people
# dwellsBedDF_1 = pd.merge(dwellsBedDF, mbperthDF, on=['SA1'], how='left')
dwellsBedDF_1 = dwellsBedDF

# merge columns
dwellsBedDF_1 = mergeColumns(dwellsBedDF_1, '5 or more bedrooms', [
        '5 bedrooms',
        '6 bedrooms',
        '7 bedrooms',
        '8 bedrooms',
        '9 bedrooms',
        '10 bedrooms',
        '11 bedrooms',
        '12 bedrooms',
        '13 bedrooms',
        '14 bedrooms',
        '15 bedrooms',
        '16 bedrooms',
        '17 bedrooms',
        '18 bedrooms',
        '19 bedrooms',
        '20 bedrooms',
        '21 bedrooms',
        '22 bedrooms',
        '23 bedrooms',
        '24 bedrooms',
        '25 bedrooms',
        '26 bedrooms',
        '27 bedrooms',
        '28 bedrooms',
        '29 bedrooms',
        '30 or more bedrooms',
])

# rename columns
dwellsBedDF_1 = dwellsBedDF_1.rename(columns={
        'None (includes bedsitters)': 'bedroom_0',
        '1 bedroom': 'bedroom_1',
        '2 bedrooms': 'bedroom_2',
        '3 bedrooms': 'bedroom_3',
        '4 bedrooms': 'bedroom_4',
        '5 or more bedrooms': 'bedroom_5',
        'Not stated': 'bedroom_6',
        'Not applicable': 'bedroom_7'
})

# order the columns
# dwellsBedDF_1 = moveCol(dwellsBedDF_1, 'SA4', 1)
dwellsBedDF_1 = moveCol(dwellsBedDF_1, 'bedroom_0', 1)
dwellsBedDF_1 = moveCol(dwellsBedDF_1, 'bedroom_1', 2)
dwellsBedDF_1 = moveCol(dwellsBedDF_1, 'bedroom_2', 3)
dwellsBedDF_1 = moveCol(dwellsBedDF_1, 'bedroom_3', 4)
dwellsBedDF_1 = moveCol(dwellsBedDF_1, 'bedroom_4', 5)
dwellsBedDF_1 = moveCol(dwellsBedDF_1, 'bedroom_5', 6)
dwellsBedDF_1 = moveCol(dwellsBedDF_1, 'bedroom_6', 7)
dwellsBedDF_1 = moveCol(dwellsBedDF_1, 'bedroom_7', 8)

RandomHead(dwellsBedDF_1, 5)

,SA1,bedroom_0,bedroom_1,bedroom_2,bedroom_3,bedroom_4,bedroom_5,bedroom_6,bedroom_7
1795,5109912,0,0,0,37,185,9,9,0
1796,5109913,0,0,0,29,83,6,8,0
1797,5109914,0,0,0,0,0,0,0,0
1798,5109915,0,3,4,78,103,10,10,0
1799,5109916,0,0,0,33,67,3,5,0


## Rent

In [18]:
def readDwellingsRent2011():
    # columns to use
    columns = {
        'SA1': int,
        'Nil payments': int,
        '$1-$74': int,
        '$75-$99': int, 
        '$100-$124': int,
        '$125-$149': int,
        '$150-$174': int,
        '$175-$199': int,
        '$200-$224': int,
        '$225-$249': int,
        '$250-$274': int,
        '$275-$299': int,
        '$300-$324': int,
        '$325-$349': int,
        '$350-$374': int,
        '$375-$399': int,
        '$400-$424': int,
        '$425-$449': int,
        '$450-$549': int,
        '$550-$649': int,
        '$650 and over': int,
        'Not stated': int,
        'Not applicable': int,
    }
    # read the data
    df = pd.read_csv(dataPath + 'in_TB_OPD_RNT_SA1.csv', usecols = columns.keys(), dtype = columns)
    # rename
    df = df.rename(columns={
        'Nil payments': 'rent_1',
        '$1-$74': 'rent_2',
        '$75-$99': 'rent_3', 
        '$100-$124': 'rent_4',
        '$125-$149': 'rent_5',
        '$150-$174': 'rent_6',
        '$175-$199': 'rent_7',
        '$200-$224': 'rent_8',
        '$225-$249': 'rent_9',
        '$250-$274': 'rent_10',
        '$275-$299': 'rent_11',
        '$300-$324': 'rent_12',
        '$325-$349': 'rent_13',
        '$350-$374': 'rent_14',
        '$375-$399': 'rent_15',
        '$400-$424': 'rent_16',
        '$425-$449': 'rent_17',
        '$450-$549': 'rent_18',
        '$550-$649': 'rent_19',
        '$650 and over': 'rent_20',
        'Not stated': 'rent_21',
        'Not applicable': 'rent_22',
    })

    return df
    
dwellsRntDF_1 = readDwellingsRent2011()
dwellsRntDF_1.head(5)

,SA1,rent_1,rent_2,rent_3,rent_4,rent_5,rent_6,rent_7,rent_8,rent_9,...,rent_13,rent_14,rent_15,rent_16,rent_17,rent_18,rent_19,rent_20,rent_21,rent_22
0,5103001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,176
1,5103002,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,3,3,11,0,94
2,5103003,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,5103004,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5103005,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
# assign values from MB_WA to people = correlate regions to people
# dwellsRntDF_1 = pd.merge(dwellsRntDF, mbperthDF, on=['SA1'], how='left')
# order the columns
# dwellsRntDF_1 = moveCol(dwellsRntDF_1, 'SA4', 1)

# RandomHead(dwellsRntDF_1, 5)

## Mortgage

In [20]:
def readDwellingsMortgage2011():
    # columns to use
    columns = {
        'SA1': int,
        'Nil repayments': int,
        '$1-$149': int,
        '$150-$299': int,
        '$300-$449': int,
        '$450-$599': int,
        '$600-$799': int,
        '$800-$999': int,
        '$1,000-$1,199': int,
        '$1,200-$1,399': int,
        '$1,400-$1,599': int,
        '$1,600-$1,799': int,
        '$1,800-$1,999': int,
        '$2,000-$2,199': int,
        '$2,200-$2,399': int,
        '$2,400-$2,599': int,
        '$2,600-$2,999': int,
        '$3,000-$3,999': int,
        '$4,000-$4,999': int,
        '$5000 and over': int,
        'Not stated': int,
        'Not applicable': int,
    }
    # read the data
    df = pd.read_csv(dataPath + 'in_TB_OPD_MRG_SA1.csv', usecols = columns.keys(), dtype = columns)
    # rename
    df = df.rename(columns={
        'Nil repayments': 'mortgage_1',
        '$1-$149': 'mortgage_2',
        '$150-$299': 'mortgage_3',
        '$300-$449': 'mortgage_4',
        '$450-$599': 'mortgage_5',
        '$600-$799': 'mortgage_6',
        '$800-$999': 'mortgage_7',
        '$1,000-$1,199': 'mortgage_8',
        '$1,200-$1,399': 'mortgage_9',
        '$1,400-$1,599': 'mortgage_10',
        '$1,600-$1,799': 'mortgage_11',
        '$1,800-$1,999': 'mortgage_12',
        '$2,000-$2,199': 'mortgage_13',
        '$2,200-$2,399': 'mortgage_14',
        '$2,400-$2,599': 'mortgage_15',
        '$2,600-$2,999': 'mortgage_16',
        '$3,000-$3,999': 'mortgage_17',
        '$4,000-$4,999': 'mortgage_18',
        '$5000 and over': 'mortgage_19',
        'Not stated': 'mortgage_20',
        'Not applicable': 'mortgage_21',
    })
    return df
    
dwellsMrgDF_1 = readDwellingsMortgage2011()
dwellsMrgDF_1.head(5)

,SA1,mortgage_1,mortgage_2,mortgage_3,mortgage_4,mortgage_5,mortgage_6,mortgage_7,mortgage_8,mortgage_9,...,mortgage_12,mortgage_13,mortgage_14,mortgage_15,mortgage_16,mortgage_17,mortgage_18,mortgage_19,mortgage_20,mortgage_21
0,5103001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3,0,172
1,5103002,0,0,0,0,0,0,0,0,0,...,4,0,0,0,0,4,3,10,5,76
2,5103003,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,5103004,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
4,5103005,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Vehicles

In [21]:
def readDwellingsVehicles2011():
    # columns to use
    columns = {
        'SA1': int,
        'No motor vehicles': int,
        '1 motor vehicle': int,
        '2 motor vehicles': int,
        '3 motor vehicles': int,
        '4 motor vehicles': int,
        '5 motor vehicles': int,
        '6 motor vehicles': int,
        '7 motor vehicles': int,
        '8 motor vehicles': int,
        '9 motor vehicles': int,
        '10 motor vehicles': int,
        '11 motor vehicles': int,
        '12 motor vehicles': int,
        '13 motor vehicles': int,
        '14 motor vehicles': int,
        '15 motor vehicles': int,
        '16 motor vehicles': int,
        '17 motor vehicles': int,
        '18 motor vehicles': int,
        '19 motor vehicles': int,
        '20 motor vehicles': int,
        '21 motor vehicles': int,
        '22 motor vehicles': int,
        '23 motor vehicles': int,
        '24 motor vehicles': int,
        '25 motor vehicles': int,
        '26 motor vehicles': int,
        '27 motor vehicles': int,
        '28 motor vehicles': int,
        '29 motor vehicles': int,
        '30 or more motor vehicles': int,
        'Not stated': int,
        'Not applicable': int,
    }
    # read the data
    df = pd.read_csv(dataPath + 'in_TB_OPD_VEH_SA1.csv', usecols = columns.keys(), dtype = columns)

    return df
    
dwellsVehDF = readDwellingsVehicles2011()
dwellsVehDF.head(5)

,SA1,No motor vehicles,1 motor vehicle,2 motor vehicles,3 motor vehicles,4 motor vehicles,5 motor vehicles,6 motor vehicles,7 motor vehicles,8 motor vehicles,...,23 motor vehicles,24 motor vehicles,25 motor vehicles,26 motor vehicles,27 motor vehicles,28 motor vehicles,29 motor vehicles,30 or more motor vehicles,Not stated,Not applicable
0,5103001,45,100,21,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,18,0
1,5103002,0,25,58,25,12,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5103003,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,5103004,0,5,0,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5103005,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
# assign values from MB_WA to people = correlate regions to people
# dwellsVehDF_1 = pd.merge(dwellsVehDF, mbperthDF, on=['SA1'], how='left')
dwellsVehDF_1 = dwellsVehDF

# merge columns
dwellsVehDF_1 = mergeColumns(dwellsVehDF_1, '4 or more motor vehicles', [
        '4 motor vehicles',
        '5 motor vehicles',
        '6 motor vehicles',
        '7 motor vehicles',
        '8 motor vehicles',
        '9 motor vehicles',
        '10 motor vehicles',
        '11 motor vehicles',
        '12 motor vehicles',
        '13 motor vehicles',
        '14 motor vehicles',
        '15 motor vehicles',
        '16 motor vehicles',
        '17 motor vehicles',
        '18 motor vehicles',
        '19 motor vehicles',
        '20 motor vehicles',
        '21 motor vehicles',
        '22 motor vehicles',
        '23 motor vehicles',
        '24 motor vehicles',
        '25 motor vehicles',
        '26 motor vehicles',
        '27 motor vehicles',
        '28 motor vehicles',
        '29 motor vehicles',
        '30 or more motor vehicles',
])

# rename columns
dwellsVehDF_1 = dwellsVehDF_1.rename(columns={
        'No motor vehicles': 'vehicles_0',
        '1 motor vehicle': 'vehicles_1',
        '2 motor vehicles': 'vehicles_2',
        '3 motor vehicles': 'vehicles_3',
        '4 or more motor vehicles': 'vehicles_4',
        'Not stated': 'vehicles_5',
        'Not applicable': 'vehicles_6',
})

# order the columns
dwellsVehDF_1 = moveCol(dwellsVehDF_1, 'vehicles_0', 1)
dwellsVehDF_1 = moveCol(dwellsVehDF_1, 'vehicles_1', 2)
dwellsVehDF_1 = moveCol(dwellsVehDF_1, 'vehicles_2', 3)
dwellsVehDF_1 = moveCol(dwellsVehDF_1, 'vehicles_3', 4)
dwellsVehDF_1 = moveCol(dwellsVehDF_1, 'vehicles_4', 5)
dwellsVehDF_1 = moveCol(dwellsVehDF_1, 'vehicles_5', 6)
dwellsVehDF_1 = moveCol(dwellsVehDF_1, 'vehicles_6', 7)

RandomHead(dwellsVehDF_1, 5)

,SA1,vehicles_0,vehicles_1,vehicles_2,vehicles_3,vehicles_4,vehicles_5,vehicles_6
2355,5112325,59,164,73,22,3,33,0
2356,5112326,40,119,45,12,0,39,0
2357,5112327,42,95,44,8,3,26,0
2358,5112328,44,145,42,8,0,21,0
2359,5112329,27,90,32,3,0,44,0


## One dwelling file to rule them all

In [23]:
# merge the files
dwellsCombinedDF_1 = pd.merge(dwellsTotalDF_1, dwellsStructureDF_1, on=['SA1'], how='left') # count & structure
dwellsCombinedDF_2 = pd.merge(dwellsCombinedDF_1, dwellsBedDF_1, on=['SA1'], how='left') # & beds
dwellsCombinedDF_3 = pd.merge(dwellsCombinedDF_2, dwellsRntDF_1, on=['SA1'], how='left') # & rent
dwellsCombinedDF_4 = pd.merge(dwellsCombinedDF_3, dwellsMrgDF_1, on=['SA1'], how='left') # & mortgage
dwellsCombinedDF_5 = pd.merge(dwellsCombinedDF_4, dwellsVehDF_1, on=['SA1'], how='left') # & vehicles

# new file
masterDwellDF = dwellsCombinedDF_5

# sum seperate properties
masterDwellDF['structure_total'] = masterDwellDF.loc[:,'structure_1':'structure_6'].sum(axis = 1)
masterDwellDF['bedroom_total'] = masterDwellDF.loc[:,'bedroom_0':'bedroom_7'].sum(axis = 1)
masterDwellDF['rent_total'] = masterDwellDF.loc[:,'rent_1':'rent_22'].sum(axis = 1)
masterDwellDF['mortgage_total'] = masterDwellDF.loc[:,'mortgage_1':'mortgage_21'].sum(axis = 1)
masterDwellDF['vehicles_total'] = masterDwellDF.loc[:,'vehicles_0':'vehicles_6'].sum(axis = 1)

RandomHead(dwellsCombinedDF_5, 5)

,SA1,SA4,Count,structure_1,structure_2,structure_3,structure_4,structure_5,structure_6,bedroom_0,...,vehicles_2,vehicles_3,vehicles_4,vehicles_5,vehicles_6,structure_total,bedroom_total,rent_total,mortgage_total,vehicles_total
825,5105953,49,156,156,0,0,0,0,0,0,...,76,30,8,12,0,156,158,144,157,154
826,5105954,49,129,122,9,0,0,0,0,0,...,48,17,9,3,0,131,127,140,121,133
827,5105955,49,80,84,0,0,0,0,0,0,...,37,10,3,3,0,84,84,75,89,82
828,5105956,49,182,177,0,0,0,0,0,0,...,85,20,3,17,0,177,173,182,185,176
829,5105957,49,109,100,16,0,0,0,0,0,...,41,11,6,5,0,116,109,109,102,110


In [24]:
masterDwellDF.to_csv(dataPath+'out_TB_opdBySA1_2011.csv', index=False)
print("CSV created.")

CSV created.
